# Reinforcement Learning
Prof. Milica Gašić

### Monte Carlo and Temporal Difference Prediction



#### Implementation

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import rl_tests
from random import random
import matplotlib
import matplotlib.pyplot as plt


We have implemented the environment for generating random walk episodes below. 

In [ ]:
def random_walk_rollout(n_states=5, left_reward=0, right_reward=1):
    terminal_states = [0, n_states-1]
    s = n_states//2 # start in the middle
    e = [s]   # episode
    while s not in terminal_states:
        # do random action 'right' or 'left'
        s += 1 if random() > 0.5 else -1     # next state
        if s == n_states - 1:
            r = right_reward
        elif s == 0:
            r = left_reward
        else:
            r = 0
        e.extend([float(r), s])
    return e

Your task is to implement value iteration for computing value estimates for the episodes. Start with implementing the update per episode.

In [ ]:
def MC_step_inc(V, e, gamma, alpha):
    #######################################################################
    # TODO  implement monte carlo prediction update for one episode       #
    # update the variable V                                               #
    #######################################################################
    
    #######################################################################
    # End of your code.                                                   #
    #######################################################################

    



def TD_step(V, e, gamma, alpha):
    #######################################################################
    # TODO  implement TD prediction update for one episode                #
    # update the variable V                                               #
    #######################################################################
    
    #######################################################################
    # End of your code.                                                   #
    #######################################################################

In [ ]:
def test_random_walk_updates():
    # Setup a fixed context for reproducible testing
    # 5 states total: 0 (Left Term), 1, 2, 3, 4 (Right Term)
    n_states = 5
    gamma = 0.9
    alpha = 0.1
    # A fixed episode: Start at 2, go Left(1), Right(2), Right(3), Right(4-Term)
    # Format: [s, r, s, r, s, r, s, r, s]
    # Note: The rewards are 0 unless hitting terminal state 4 (r=1)
    # fixed_episode = [[2, 0.0, 1, 0.0, 2, 0.0, 3, 1.0, 4], [2, 0.0, 1, 0.0, 0]]

    # ---------------------------------------------------------
    yield 'MC_step_inc()'
    # ---------------------------------------------------------
    # Expected calculation for MC (Every-visit):
    # t=6 (s=3, r=1): G=1.0. V[3] += 0.1*(1.0 - 0.5) -> 0.55
    # t=4 (s=2, r=0): G=0.9. V[2] += 0.1*(0.9 - 0.5) -> 0.54
    # t=2 (s=1, r=0): G=0.81. V[1] += 0.1*(0.81 - 0.5) -> 0.531
    # t=0 (s=2, r=0): G=0.729. V[2] += 0.1*(0.729 - 0.54) -> 0.5589
    # Sum = 0.531 + 0.5589 + 0.55 = 1.6399
    for fixed_episode, expected_mc_sum in zip([[2, 0.0, 1, 0.0, 2, 0.0, 3, 1.0, 4], [2, 0.0, 1, 0.0, 0]], [1.6399, 1.4000]):
        # Initialize V with 0.5 for non-terminals
        V_mc = np.array([0.0, 0.5, 0.5, 0.5, 0.0])

        MC_step_inc(V_mc, fixed_episode, gamma, alpha)

        # Verify shape/type using your framework's helper
        if (yield from rl_tests.check_numpy_array(V_mc, name='V', shape=(n_states,), dtype=np.floating)):
            v_sum = np.sum(V_mc)
            yield np.isclose(v_sum, expected_mc_sum, atol=1e-5), \
                f'The MC value updates are incorrect. Expected sum ~{expected_mc_sum:.4f}, got {v_sum:.4f}'
        yield None


    # ---------------------------------------------------------
    yield 'TD_step()'
    # ---------------------------------------------------------
    # Expected calculation for TD:
    # t=0 (s=2->1): Tgt=0+0.9*0.5=0.45. V[2] += 0.1*(0.45-0.5) -> 0.495
    # t=2 (s=1->2): Tgt=0+0.9*0.495=0.4455. V[1] += 0.1*(0.4455-0.5) -> 0.494555
    # t=4 (s=2->3): Tgt=0+0.9*0.5=0.45. V[2] += 0.1*(0.45-0.495) -> 0.4905
    # t=6 (s=3->4): Tgt=1+0.9*0=1.0. V[3] += 0.1*(1.0-0.5) -> 0.55
    # Sum = 0.494555 + 0.4905 + 0.55 = 1.535055
    for fixed_episode, expected_td_sum in zip([[2, 0.0, 1, 0.0, 2, 0.0, 3, 1.0, 4], [2, 0.0, 1, 0.0, 0]], [1.535055, 1.4450]):

        # Reset V for TD test
        V_td = np.array([0.0, 0.5, 0.5, 0.5, 0.0])

        TD_step(V_td, fixed_episode, gamma, alpha)

        if (yield from rl_tests.check_numpy_array(V_td, name='V', shape=(n_states,), dtype=np.floating)):
            v_sum = np.sum(V_td)
            yield np.isclose(v_sum, expected_td_sum, atol=1e-5), \
                f'The TD value updates are incorrect. Expected sum ~{expected_td_sum:.4f}, got {v_sum:.4f}'
        yield None

# Run the tests
rl_tests.run_tests(test_random_walk_updates())

We would like to track how our value estimates converge over time. 
We will rollout a number of episodes, and update our estimates using the function implemented above.
We iterate this episode sampling and evaluation maxiter amount of times
We would like to plot the convergence of each method, so we keep the error to be plotted later

In [ ]:

def RMSError(v0, v1):
    return np.sqrt(((v0-v1)**2).mean())

def mc_compute_v(n_episodes, n_states, alpha, maxiter):
    
    rms = np.zeros(n_episodes) #list to collect rms for each episode

    # iterate maxiter amount of times
    for k in range(maxiter):
        # roll out episodes
        episodes = []
        for _ in range(n_episodes):
            episodes.append(random_walk_rollout(n_states = n_states))

        #initialize containers
        v = np.zeros(n_states)
        
        #######################################################################
        # TODO loop through the episodes and                                  #
        # call the episode-wise MC update you have implemented above          #
        # for each iteraction, accummulate the rms per episode                #
        #######################################################################

        
            
        #######################################################################
        # End of your code.                                                   #
        #######################################################################
    
    rms /= maxiter # average the rms
    
    return v, rms
    

def td_compute_v(n_episodes, n_states, alpha, maxiter):
    
    rms = np.zeros(n_episodes) #list to collect rms for each episode

    # iterate maxiter amount of times
    for k in range(maxiter):
        # roll out episodes
        episodes = []
        for _ in range(n_episodes):
            episodes.append(random_walk_rollout(n_states = n_states))
            
        v = np.zeros(n_states)
        
        #######################################################################
        # TODO call the episode-wise TD update you have implemented above     #
        # and maintain a list tracking the RMS error of the estimate          #
        # for each iteraction, accummulate the rms per episode                #
        #######################################################################

        

        #######################################################################
        # End of your code.                                                   #
        #######################################################################
    
    rms /= maxiter # average the rms
    
    return v, rms
    

Call the functions above to answer the questions:
* How does MC and TD differs?
* What is the influence of alpha on the convergence?

In [ ]:
episodes = []

gamma = 1.0
n_states = 5
n_episodes = 100
maxiter = 100

td_alphas=[0.15, 0.2, 0.25]
mc_alphas=[0.01, 0.02, 0.03, 0.04]

v_true = np.linspace(0, 1, num=n_states)
v_true[-1] = 0

# rollout episodes
#for _ in range(n_episodes):
#    episodes.append(random_walk_rollout(n_states = n_states))

# value estimates with MC:
print("\n====== MC ======\n")
for alpha in mc_alphas:
    print(f"alpha:{alpha}")
    v, rms = mc_compute_v(n_episodes, n_states, alpha, maxiter)
    plt.plot(rms, '--', label=f'MC {alpha}')
    print(v)

# value estimates with TD:
print("\n====== TD ======\n")
for alpha in td_alphas:
    print(f"alpha:{alpha}")
    v, rms = td_compute_v(n_episodes, n_states, alpha, maxiter)
    plt.plot(rms, label=f'TD {alpha}')
    print(v)

plt.xlabel('walks')
plt.ylabel('RMS error')
plt.legend()
plt.show()